In [6]:
def Ulam(a, b, n=100):
    seq = [a, b]
    seen = set(seq)
    to_check = max(a, b) + 1
    for _ in range(n):
        while True:
            # Try to check
            works = True
            sum_cnts = 0

            for i in range(1, (to_check+1) // 2):
                comp = to_check - i
                if i in seen and comp in seen:
                    sum_cnts += 1

            if sum_cnts == 1:
                seq.append(to_check)
                seen.add(to_check)
                to_check += 1
                break
            else:
                to_check += 1
    return seq[0:n]

In [7]:
def diff(s):
    return [s[i + 1] - s[i] for i in range(len(s) - 1)]


In [15]:
Ulam(2, 5, 20)

[2, 5, 7, 9, 11, 12, 13, 15, 19, 23, 27, 29, 35, 37, 41, 43, 45, 49, 51, 55]

In [ ]:
diff(Ulam(2, 5, 20))

[3, 2, 2, 2, 1, 1, 2, 4, 4, 4, 2, 6, 2, 4, 2, 2, 4, 2, 4]

In [9]:
Ulam(2, 5, 3000)[-1]

11789

In [10]:
def solve(a, b, k):
    # For Ulam(2, 2k + 1) {k >= 2}, there is periodicity.
    assert(a == 2 and b & 1 and b >= 5)

    to_try, times_to_check, max_p_len = 2_000, 5, 300
    seq = Ulam(a, b, n=to_try) 
    d = diff(seq)
    rdiff = list(reversed(d))
    # Try different period lengths until we find 5 in a row. From this extrapolate to k = 10^11
    # print(d)
    p_len, inc = None, None
    while True:
        for p_len in range(1, max_p_len):
            w = True
            for q in range(times_to_check):
                # print(rdiff[q * p_len:(q+1) * p_len], rdiff[(q+1) * p_len:(q+2) * p_len])
                if rdiff[q * p_len:(q+1) * p_len] != rdiff[(q+1) * p_len:(q+2) * p_len]:
                    w = False
            if w:
                inc = sum(rdiff[q * p_len:(q+1) * p_len])
                break
        if inc == None:
            # If didnt find anything mult all params and retry
            print("Period not found under to_try: {} and max_p_len: {}".format(to_try, max_p_len))
            to_try *= 2
            max_p_len *= 2
            seq = Ulam(a, b, n=to_try) 
            d = diff(seq)
            rdiff = list(reversed(d))
        else:
            print("Found p_len: {} and inc: {}".format(p_len, inc))
            break
    final_el = seq[-1]
    curr_pos = len(seq)
    times = (k - curr_pos) // p_len
    final_el, curr_pos = final_el + inc * times, curr_pos + times * p_len
    i = len(d) - p_len
    while curr_pos != k:
        final_el += d[i]
        curr_pos += 1
        i += 1
    return final_el


In [14]:
tot = 0
for n in range(2, 3 + 1):
    print("Solving U(2, {})".format(2 * n + 1))
    T = solve(2, 2 * n + 1, 10**11)
    tot += T
    print("Got", T)
print(tot)

Solving U(2, 5)
Found p_len: 32 and inc: 126
Got 393749999981
Solving U(2, 7)
Found p_len: 26 and inc: 126
Got 484615384605
878365384586


In [132]:
import heapq

def fast_ulam(u, v, n, active_window=2000):
    """
    Generate n terms of the (u,v)-Ulam sequence.
    
    Parameters:
      u, v : starting integers with u < v
      n    : number of Ulam numbers to generate
      active_window : maximum size of the active set used for candidate updates
      
    Returns:
      A list containing the first n Ulam numbers.
    """
    if u >= v:
        raise ValueError("u must be less than v")
    
    # Initial Ulam sequence
    U = [u, v]
    
    # active set: heuristically, keep only the last few Ulam numbers that are likely to contribute
    active = [u, v]
    
    # candidate dictionary: candidate sum -> count of representations
    candidate = {}
    # heap for candidate sums (to get the minimum candidate quickly)
    candidate_heap = []
    
    # Initialize with the only candidate from the first two numbers
    s = u + v
    candidate[s] = 1
    heapq.heappush(candidate_heap, s)
    
    while len(U) < n:
        # Find the next Ulam number: the smallest candidate > last Ulam number with unique representation.
        next_ulam = None
        while candidate_heap:
            s = heapq.heappop(candidate_heap)
            # Skip if the candidate is not strictly greater than the last Ulam or if its count is not 1.
            if s <= U[-1] or candidate[s] != 1:
                continue
            next_ulam = s
            break
        
        if next_ulam is None:
            raise RuntimeError("No valid candidate found. Consider increasing candidate range.")
        
        # Append the next Ulam number
        U.append(next_ulam)
        
        # Update candidate sums using only the active window:
        for x in active:
            newsum = x + next_ulam
            # Update count for this sum
            candidate[newsum] = candidate.get(newsum, 0) + 1
            # Push to heap if it’s a new candidate sum
            if candidate[newsum] == 1:
                heapq.heappush(candidate_heap, newsum)
        
        # Add the new Ulam number to the active set
        active.append(next_ulam)
        # Prune the active set to keep it within the desired window size.
        if len(active) > active_window:
            active = active[-active_window:]
    
    return U

ulam_seq = fast_ulam(2, 19, 3000)[-1]
print(ulam_seq)


79460


In [ ]:
def ulam(a, b, n):
  upper = n
  upper *= 2
  V = [0]*upper
  W = [1]*upper
  V[a - 1] = 1
  V[b - 1] = 1
  W[a - 1] = 0
  W[b - 1] = 0
  k=2

  while True and k < upper/2:
    V[k:2*k-1] = [((not a) and b and c) or (a and (not b) and (not c)) or (a and (not b) and c) for a,b,c in zip(V[k:2*k-1], V[0:k-1], W[k:2*k-1])]
    W[k:2*k-1] = [(not a) and b for a,b in zip(V[0:k-1], W[k:2*k-1])]
    # print(V)
    L = [j for j in range(k+1,upper) if V[j-1] == 1]
    if L == []:
      break
    else:
      k = min(L)
  ls = [i+1 for i in range(0, len(V)) if V[i]==1]
  ls = ls[0:len(ls)//2]
  return ls

In [144]:
def ulam(a, b, n):
  upper = n
  upper *= 2
  V = [0]*upper
  W = [1]*upper
  V[a - 1] = 1
  V[b - 1] = 1
  W[a - 1] = 0
  W[b - 1] = 0
  k=2

  while True and k < upper/2:
    V[k:2*k-1] = [((not a) and b and c) or (a and (not b) and (not c)) or (a and (not b) and c) for a,b,c in zip(V[k:2*k-1], V[0:k-1], W[k:2*k-1])]
    W[k:2*k-1] = [(not a) and b for a,b in zip(V[0:k-1], W[k:2*k-1])]
    # print(V)
    L = [j for j in range(k+1,upper) if V[j-1] == 1]
    if L == []:
      break
    else:
      k = min(L)
  ls = [i+1 for i in range(0, len(V)) if V[i]==1]
  ls = ls[0:len(ls)//2]
  return ls
def solve(a, b, k):
    # For Ulam(2, 2k + 1) {k >= 2}, there is periodicity.
    assert(a == 2 and b & 1 and b >= 5)

    to_try, times_to_check, max_p_len = 500, 5, 80
    seq = ulam(a, b, n=to_try) 
    d = diff(seq)
    rdiff = list(reversed(d))
    # Try different period lengths until we find 5 in a row. From this extrapolate to k = 10^11
    # print(d)
    p_len, inc = None, None
    while True:
        last_p_len = 1
        for p_len in range(last_p_len, max_p_len + 1):
            w = True
            for q in range(times_to_check):
                # print(rdiff[q * p_len:(q+1) * p_len], rdiff[(q+1) * p_len:(q+2) * p_len])
                if rdiff[q * p_len:(q+1) * p_len] != rdiff[(q+1) * p_len:(q+2) * p_len]:
                    w = False
                    break
            if w:
                inc = sum(rdiff[q * p_len:(q+1) * p_len])
                break
        if inc == None:
            # If didnt find anything mult all params and retry
            print("Period not found under to_try: {} and max_p_len: {}".format(to_try, max_p_len))
            last_p_len = max_p_len
            to_try *= 2
            max_p_len *= 2
            seq = ulam(a, b, n=to_try) 
            d = diff(seq)
            rdiff = list(reversed(d))
        else:
            print("Found p_len: {} and inc: {}".format(p_len, inc))
            break
    final_el = seq[-1]
    curr_pos = len(seq)
    times = (k - curr_pos) // p_len
    final_el, curr_pos = final_el + inc * times, curr_pos + times * p_len
    i = len(d) - p_len
    while curr_pos != k:
        final_el += d[i]
        curr_pos += 1
        i += 1
    return final_el

tot = 0
for n in range(2, 10 + 1):
    print("Solving U(2, {})".format(2 * n + 1))
    tot += solve(2, 2 * n + 1, 10**11)
print(tot)

Solving U(2, 5)
Period not found under to_try: 500 and max_p_len: 80
Period not found under to_try: 1000 and max_p_len: 160
Found p_len: 32 and inc: 126
Solving U(2, 7)
Period not found under to_try: 500 and max_p_len: 80
Period not found under to_try: 1000 and max_p_len: 160
Found p_len: 26 and inc: 126
Solving U(2, 9)
Period not found under to_try: 500 and max_p_len: 80
Period not found under to_try: 1000 and max_p_len: 160
Period not found under to_try: 2000 and max_p_len: 320
Period not found under to_try: 4000 and max_p_len: 640
Period not found under to_try: 8000 and max_p_len: 1280


KeyboardInterrupt: 

In [ ]:
def ulam_bit(a, b, n):
    v = [0 for _ in range(n)]
    v[a - 1], v[b - 1] = 1, 1
    k = b

    while k < n:
        for i in range(min(k - 1, n - k)):
            v[i+k] += v[i] == 1
        k += 1
        while k - 1 < n and v[k - 1] != 1:
            k += 1

    return [i+1 for i in range(n) if v[i] == 1]
U = ulam_bit(2, 19, 100_000)
len(U)

In [169]:

U = ulam_bit(2, 19, 100_000)
len(U)

KeyboardInterrupt: 